# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import re
import vk_api
from datetime import datetime, timedelta

In [2]:
KEYWORDS = ['python', 'парсинг', 'nlp']

In [3]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
#soup

In [4]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
#posts

In [5]:
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)

post 521134
post 328784
post 521244
post 521174
post 521010
post 521178
post 521234
post 521038
post 520864
post 520632
post 521166
post 521226
post 520972
post 521222
post 521206
post 521144
post 521204
post 520662
post 521212
post 521202


In [6]:
for post in posts:
    post_id = post.parent.attrs.get('id')
    time.sleep(0.2)
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    title_element = post.find('a', class_='post__title_link')
    date = post.find('span', class_='post__time')
    res = requests.get(title_element.attrs.get('href'))
    soup = BeautifulSoup(res.text)
    ke = soup.find('meta', attrs={'name':'keywords'})['content']
    ke1 = ke.lower()
    for j in KEYWORDS:
        if j in ke1:
            print(date.text, title_element.text, title_element.attrs.get('href'), ke)

сегодня в 17:00 Самый точный прогноз погоды: бот для Telegram на облачных функциях https://habr.com/ru/company/selectel/blog/521038/ atis, metar, volmet, taf, telegram, bot, python, serverless
сегодня в 16:28 Более эффективное предварительное обучение NLP моделей с ELECTRA https://habr.com/ru/post/521166/ NLP, ELECTRA


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [7]:
for post in posts:
    post_id = post.parent.attrs.get('id')
    time.sleep(0.3)
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    title_element = post.find('a', class_='post__title_link')
    date = post.find('span', class_='post__time')
    res = requests.get(title_element.attrs.get('href'))
    soup = BeautifulSoup(res.text)
    if soup.find('div', class_='post__text post__text-html post__text_v1'):
        text_article = soup.find('div', class_='post__text post__text-html post__text_v1').text
    elif soup.find('div', class_='post__text post__text-html post__text_v2'):
        text_article = soup.find('div', class_='post__text post__text-html post__text_v2').text
    elif soup.find('div', class_='post__text post__text_v2'):
        text_article = soup.find('div', class_='post__text post__text_v2').text    
    ta = text_article.lower().strip()
    ta1 = re.split(r'\W+', ta)
    ke = soup.find('meta', attrs={'name':'keywords'})['content']
    ke1 = ke.lower()
    for j in KEYWORDS:
        if j in ke1:
            print(date.text, title_element.text, title_element.attrs.get('href'))
            print(ke)
            print('\n')
        else:
            if j in ta1:
                print(date.text, title_element.text, title_element.attrs.get('href'))
                print(ke)
                print(text_article[:300])
                print('\n')

сегодня в 17:09 Университет или буткемп? Разбираемся, что лучше https://habr.com/ru/company/skillfactory/blog/521178/
skillfactory, буткемп, университет, bootcamp


Совсем недавно мы запустили буткемпы по Data Science и Data Analytics и хотели бы поделиться с хабражителями материалом, в котором автор сравнивает этот популярный на Западе формат образования с классическим университетским. Приятного чтения.

От автора оригинала.

Я поступил в Корнельский унив


сегодня в 17:00 Самый точный прогноз погоды: бот для Telegram на облачных функциях https://habr.com/ru/company/selectel/blog/521038/
atis, metar, volmet, taf, telegram, bot, python, serverless


сегодня в 16:28 Более эффективное предварительное обучение NLP моделей с ELECTRA https://habr.com/ru/post/521166/
NLP, ELECTRA


сегодня в 16:15 PiAlert V1 на страже безопасности серверов https://habr.com/ru/company/ruvds/blog/520972/
Raspberry Pi, DIY, системное администрирование
Однажды у меня появилась идея. Эта идея воплотилась в проек

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [8]:
email = ['***', '***']

In [9]:
headers = {
'Vaar-Version': '0',
'Vaar-Header-App-Product': 'hackcheck-web-avast',
'Content-Type': 'application/json',
}

In [10]:
data = '{"emailAddresses":["***", "***"]}'

In [11]:
response = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', headers=headers, data=data) 

In [12]:
k=0
for j in response.json()['summary']:
    print(j)
    for i in response.json()['summary'][j]['breaches']:
        k+=1
        print(datetime.strptime(response.json()['breaches'][str(i)]['publishDate'], '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M'))
        print(k,': ', response.json()['breaches'][str(i)]['site'])
        print(response.json()['breaches'][str(i)]['description'])
    k=0
    print('\n')

elena-mi.14@mail.ru
2020-07-23 00:00
1 :  wattpad.com
In June 2020, the online writing community Wattpad was allegedly breached. The stolen data contains usernames, IPs, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.
2019-05-16 00:00
2 :  shein.com
In June 2018, fast-fashion retailer Shein's database was allegedly breached. The stolen data contains passwords and email addresses. This breach is being privately shared on the internet.
2016-10-29 00:00
3 :  vk.com
Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. 
2019-05-23 00:00
4 :  livejournal.com
In 2017, social network LiveJournal's databas

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [13]:
group = 'netology'
token = '***'

In [14]:
user_id = ***
v=5.124

Получение данных по пользователю

In [15]:
res_users = requests.get(f'https://api.vk.com/method/users.get?user_ids={user_id}&fields=bdate, city&access_token={token}&v={v}')

In [16]:
res_users.json()

{'response': [{'id': 34389739,
   'first_name': 'Elena',
   'last_name': 'Mironenko',
   'is_closed': True,
   'can_access_closed': False,
   'bdate': '14.2.1987',
   'city': {'id': 1, 'title': 'Moscow'}}]}

Получение данных по сообществу

In [17]:
res_wall = requests.get(f'https://api.vk.com/method/wall.get?domain={group}&count=20&access_token={token}&v={v}')

In [18]:
for i in res_wall.json()['response']['items']:
    print(datetime.fromtimestamp(i['date']).strftime('%Y-%m-%d %H:%M:%S'))
    print(i['text'])
    print('\n')

2020-09-28 11:00:10
Всё ещё дарим подарки 🎁
 
Каждую неделю разыгрываем небольшие призы от Нетологии, а 5 октября подведём окончательные итоги и самому активному участнику сообщества подарим грант 20 000 рублей, которые можно потратить на любой онлайн-курс Нетологии. 
 
🥳На этой неделе мы разыграем 3 подписки на библиотеку видеокурсов Нетологии http://netolo.gy/fRQ, а также подведём итоги за месяц и определим того, кто получит главный приз. 
 
Немного о правилах и призах: 
🔹Каждую неделю несколько самых активных участников получат приз. Нельзя дважды оказаться в числе призёров: если вы заняли 1-е место в рейтинге повторно, приз переходит участнику, который находится строчкой ниже. 
🔹Самый активный участник месяца выбирается по итогам всех четырёх недель, а значит не стоит останавливаться после того, как выиграли недельный приз. 
 
Чтобы заработать баллы, ставьте лайки и пишите комментарии. Рейтинг можно посмотреть в меню сообщества. 
Подробнее о том, как начисляются баллы → vk.com/@net

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
